# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import re
from collections import Counter

class NorvigSolution():
    def __init__(self, file="big.txt"): 
        self.WORDS = Counter(self.words(open(file).read()))
        self.N = sum(self.WORDS.values())

    def words(self, text): return re.findall(r'\w+', text.lower())

    def P(self, word): 
        "Probability of `word`."
        return self.WORDS[word] / self.N

    def correction(self, word): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

My solution is based on Norvig's solution, but I will use bigram model to count word probabilities. If there is no info on the previous word, then I will use Norvig's algorithm to compute word probability.

In [2]:
class MySolution(NorvigSolution):
    def __init__(self, words_file="big.txt", bigrams_file="bigrams.txt"):
        super().__init__(words_file)

        self.BIGRAMS = self.bigrams(open(bigrams_file).readlines())
        self.BIGRAMS_N = Counter()
        for (first, second), count in self.BIGRAMS.items():
            self.BIGRAMS_N[first] += count
    
    def bigrams(self, lines):
        return Counter({(first, second) : int(count) for line in lines for count, first, second in [line.split()]})
    
    def P(self, word, prev_word=None):
        "Probability of `word` given `prev_word`."
        if prev_word is None or prev_word not in self.BIGRAMS_N:
            return super().P(word)
        return self.BIGRAMS[(prev_word, word)] / self.BIGRAMS_N[prev_word]
    
    def correction(self, word, prev_word=None): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=lambda x: self.P(x, prev_word))

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

The provided bigram dataset was used because of simplicity in implementation. The assumption that the less edits the more probable the correction is not that bad, so I have not modfied the Error Model.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

Dataset for evaluation was taken from https://www.dcs.bbk.ac.uk/~roger/corpora.html ([holbrook-tagged.dat](https://www.dcs.bbk.ac.uk/~roger/holbrook-tagged.dat)). With this dataset we can extract the misspelled word with its previous word for bigram model.

In [3]:
test_data = re.findall(r"(\w*?) <ERR targ=(.*?)> (.*?) <\/ERR>", open("holbrook-tagged.txt").read())
# test_data is a list of tuples (prev_word, correct_word, misspelled_word)
test_data[:5]

[('and', 'sister', 'siter'),
 ('My', 'sister', 'siter'),
 ('', 'goes', 'go'),
 ('out', 'sometimes', 'some times'),
 ('out', 'sometimes', 'some times')]

### Norvig's Solution test

In [4]:
norvig = NorvigSolution()
correct = 0
for (prev_word, correct_word, misspelled_word) in test_data:
    correction = norvig.correction(misspelled_word)
    if correction == correct_word:
        correct += 1

print(f"{correct}/{len(test_data)} ({round(correct / len(test_data) * 100)}%) words are corrected correctly (pleonasm, huh)")

524/2534 (21%) words are corrected correctly (pleonasm, huh)


### My Solution test

In [5]:
my = MySolution()
correct = 0
for (prev_word, correct_word, misspelled_word) in test_data:
    correction = my.correction(misspelled_word, prev_word if len(prev_word) > 0 else None)
    if correction == correct_word:
        correct += 1

print(f"{correct}/{len(test_data)} ({round(correct / len(test_data) * 100)}%) words are corrected correctly (pleonasm, huh)")

608/2534 (24%) words are corrected correctly (pleonasm, huh)


Since my solution utilizes some of the context (the previous word), it has shown better accuracy